In [ ]:
import requests

# I need a function to get historical data from Binance Api using requests lib and save it to csv file, i need to specify timerange also and from date to date

In [1]:
%pip install ccxt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 52.0 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 51.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.4 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.0/264.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.7/288.7 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 5.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import ccxt
import pandas as pd
from datetime import datetime

exchange = ccxt.binance()  # We may add API key and secret here if necessary

def fetch_historical_data(symbol, timeframe, since, until):
    start_date = exchange.parse8601(since)
    end_date = exchange.parse8601(until)
    
    historical_data = []
    while start_date < end_date:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, start_date)
        if not ohlcv:
            break
        historical_data += ohlcv
        start_date = ohlcv[-1][0] + exchange.parse_timeframe(timeframe) * 1000
        
    df = pd.DataFrame(historical_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.to_csv(f'{symbol}_historical_data.csv', index=False)
    return df

# Example usage: fetch data for BTC/USDT symbol, 1d (daily) timeframe, and from 2021-01-01 to 2021-08-01
historical_data = fetch_historical_data('BTC/USDT', '1d', '2021-01-01T00:00:00Z', '2021-08-01T00:00:00Z')
print(historical_data)

     timestamp      open      high       low     close         volume
0   2021-01-01  28923.63  29600.00  28624.57  29331.69   54182.925011
1   2021-01-02  29331.70  33300.00  28946.53  32178.33  129993.873362
2   2021-01-03  32176.45  34778.11  31962.99  33000.05  120957.566750
3   2021-01-04  33000.05  33600.00  28130.00  31988.71  140899.885690
4   2021-01-05  31989.75  34360.00  29900.00  33949.53  116049.997038
..         ...       ...       ...       ...       ...            ...
495 2022-05-11  31017.11  32162.59  27785.00  29103.94  207063.739278
496 2022-05-12  29103.94  30243.00  26700.00  29029.75  204507.263138
497 2022-05-13  29029.74  31083.37  28751.67  29287.05   97872.369570
498 2022-05-14  29287.05  30343.27  28630.00  30086.74   51095.878630
499 2022-05-15  30086.74  31460.00  29480.00  31328.89   46275.669120

[500 rows x 6 columns]
